# Imports

In [ ]:
MODEL_V2_004 = {
    "path": 'models/V2.1.004.h5',
    "mode": 'hsv',
    "test_accuracy": 0.9640,
    "test_f1": 0.9016,
    "test_loss": 0.1294,
    "test_IOU": 0.7357,
    "epochs": "10"
}
MODEL_V2_001 = {
    "path": 'models/V2.1.001.h5',
    "mode": 'rgb',
    "test_accuracy": 0.9381,
    "test_f1": 0.8202,
    "test_loss": 0.2106,
    "test_IOU": 0.3383,
    "epochs": "5"
}
MODEL_V2_003_30 = {
    "path": 'models/v2.1.003.30e.HSV.h5',
    "mode": 'hsv',
    "test_accuracy": 0.9634,
    "test_f1": 0.9023,
    "test_loss": 0.5290,
    "test_IOU": 0.1419
}
MODEL_V2_003 = {
    "path": 'models/v2.1.003.h5',
    "mode": 'hsv',
    "test_accuracy": 0.9381,
    "test_f1": 0.8202,
    "test_loss": 0.2106,
    "test_IOU": 0.3383
}
MODEL_V2_000 = {
    "path": 'models/v2.0.003.h5',
    "mode": 'rgb',
    "test_accuracy": 0.8124,
    "test_f1": 0,
    "test_loss": 0.0734,
    "test_IOU": 3.020
}
MODEL_TO_IMPORT = MODEL_V2_004 # can be either a model (dict) or a string
# MODEL_TO_IMPORT = 'best.h5' # default to...
DEFAULT_MODE = 'rgb'

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import re
from IPython import display

In [ ]:
from CNNv2 import CNN
#%run ./CNN.ipynb
#%run ./helpers.ipynb

# Load Model

In [ ]:
model = CNN(rootdir='.', window_size=608)
model.load(MODEL_TO_IMPORT if isinstance(MODEL_TO_IMPORT, str) else MODEL_TO_IMPORT['path'])
test_dir = "test_set_images/test_"

# Visualization 

In [ ]:
from skimage.color import rgb2hsv

def load_image(filename, mode = 'rgb'):
    if mode == 'hsv':
        img = rgb2hsv(mpimg.imread(filename))
    else:
        img = mpimg.imread(filename)
    return np.expand_dims(img, axis=0)
    

def visualize(idx = None, mode = 'rgb'):
    submission_filename = 'submission.csv'
    image_filenames = []
    if idx == None:
        for i in range(1, 51):
            image_filename = 'test_set_images/test_'+str(i)+'/test_' + str(i) + '.png'
            image_filenames.append(image_filename)
    elif type(idx) is int:
        image_filename = 'test_set_images/test_'+str(idx)+'/test_' + str(idx) + '.png'
        image_filenames.append(image_filename)
    else:
        for i in idx:
            image_filename = 'test_set_images/test_'+str(i)+'/test_' + str(i) + '.png'
            image_filenames.append(image_filename)
        
    foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
    # assign a label to a patch
    def patch_to_label(patch):
        df = np.mean(patch)
        if df > foreground_threshold:
            return 1
        else:
            return 0
    def mask_to_submission_strings(model, image_filename):
        img_number = int(re.search(r"\d+", image_filename).group(0))
        input_image = load_image(image_filename, mode)
        #input_image = input_image[:384, :384, :]
        #input_image = np.expand_dims(input_image, axis=0)
        Xi = model.model.predict(input_image)
        input_image = np.squeeze(input_image)
        fig, axs = plt.subplots(1,3, figsize=(16, 16))
        axs[0].imshow(input_image)
        axs[0].set_title('imput image')
        axs[1].imshow(np.squeeze(Xi))
        axs[1].set_title('real prediction')
        patch_size = 16
        Xi = np.squeeze(Xi)
        ground = Xi.copy()
        print(Xi.shape)
        for j in range(0, Xi.shape[1], patch_size):
            for i in range(0, Xi.shape[0], patch_size):
                patch = Xi[i:i + patch_size, j:j + patch_size]
                label = patch_to_label(patch)
                ground[i:i + patch_size, j:j + patch_size].fill(label)
                yield("{:03d}_{}_{},{}".format(img_number, j, i, label))
                
        axs[2].imshow(ground)
        axs[2].set_title('label prediction')
        # remove the x and y ticks
        for ax in axs:
            ax.set_xticks([])
            ax.set_yticks([])
        display.clear_output(wait=True)
        fig.suptitle('Image {0}'.format(image_filename), fontsize=16)
        
    def generate_submission(model, submission_filename, *image_filenames):
        """ Generate a .csv containing the classification of the test set. """
        with open(submission_filename, 'w') as f:
            f.write('id,prediction\n')
            for fn in image_filenames[0:]:
                f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(model, fn))
    generate_submission(model, submission_filename, *image_filenames)

In [ ]:
#Visualize prediction on input image
# Parameters idx can be:
#     int: 1-50 for a specific image
#     array: [1,2,4,6] for a specific range
#     None: run all images
visualize([1,2,3,4,6,10], mode = DEFAULT_MODE if isinstance(MODEL_TO_IMPORT, str) else MODEL_TO_IMPORT['mode'])

# Submission

In [ ]:
assert True == False, "prevent next cells execution when run all"

In [ ]:
def submission():
    submission_filename = 'submission.csv'
    image_filenames = []
    for i in range(1, 51):
        image_filename = 'test_set_images/test_'+str(i)+'/test_' + str(i) + '.png'
        image_filenames.append(image_filename)
        
    foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

    # assign a label to a patch
    def patch_to_label(patch):
        df = np.mean(patch)
        if df > foreground_threshold:
            return 1
        else:
            return 0
        
    def mask_to_submission_strings(model, image_filename):
        img_number = int(re.search(r"\d+", image_filename).group(0))
        mode = DEFAULT_MODE if isinstance(MODEL_TO_IMPORT, str) else MODEL_TO_IMPORT['mode']
        input_image = load_image(image_filename, mode)
        #input_image = np.expand_dims(loaded_img, axis=0)
        print(input_image.shape)
        Xi = model.model.predict(input_image)
        Xi = np.squeeze(Xi)
        patch_size = 16
        print("Processing " + image_filename)   
        for j in range(0, Xi.shape[1], patch_size):
            for i in range(0, Xi.shape[0], patch_size):
                patch = Xi[i:i + patch_size, j:j + patch_size]
                label = patch_to_label(patch)
                yield("{:03d}_{}_{},{}".format(img_number, j, i, label))
                        
    def generate_submission(model, submission_filename, *image_filenames):
        """ Generate a .csv containing the classification of the test set. """
        with open(submission_filename, 'w') as f:
            f.write('id,prediction\n')
            for fn in image_filenames[0:]:
                f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(model, fn))
    generate_submission(model, submission_filename, *image_filenames)

In [ ]:
submission()